In [1]:
import pandas as pd
import os
import cv2
import numpy as np
import json

In [2]:
def Detect_Office(Json,Office):

    NewList=Json['fields']
    Dict=list()
    for d in NewList:
        try:
            newDict={}
            newDict['inferText']=d['inferText']
            newDict['boundingPoly']=d['boundingPoly']
            Dict.append(newDict)
        except KeyError:
            continue

    res = [d
       for d in Dict 
       if (Office[0] == d['inferText']) or (Office == d['inferText'])]

    res = res[0]['boundingPoly']['vertices']
    Edge=max(int(d['x']) for d in res)
    return(Edge)

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(Year,Page):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
    with open(path+"Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg",'rb') as f:
         file_data = f.read()

    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json

In [3]:
Year='1934'
Showa='9'
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
os.chdir(path)
df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+Showa+'.csv')
df=df.drop(df.columns[0], axis=1)

file_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\DataFrame.json'
with open(file_path, encoding="utf-8") as f:
    dta = json.loads(f.read())

In [4]:
#Test code| Version 2#
#Show Working office#
n=2

#Extract key info of office
Row  = df.iloc[n]
ExRow= df.iloc[n-1]
Row,ExRow

(Office      文書課
 Dept      Admin
 Year          9
 Page          3
 Name: 2, dtype: object,
 Office      職員課
 Dept      Admin
 Year          9
 Page          2
 Name: 1, dtype: object)

In [5]:
#
Showa=ExRow["Year"]

Page=Row["Page"]
ExPage=ExRow["Page"]

Dept=Row["Dept"]
ExDept=ExRow["Dept"]

Office=Row["Office"]
ExOffice=ExRow["Office"]

dta[str(Year)][Dept][Office]={}
dta[str(Year)][ExDept][ExOffice]={}
dta[str(Year)][ExDept][ExOffice]["Starting_Page"]=ExPage

In [6]:
#Test code| Version 2#
#Decompose info into elements
Page=int(Row["Page"])
Office=Row["Office"]
print(Office)
ExPage=int(ExRow["Page"])
ExOffice=ExRow["Office"]


###Insert Starting page information to motherframe###
dta[Year][Dept][Office]={}
dta[Year][Dept][Office]["Starting_Page"]=Page
print(dta)

###Collect Location information###
##Read image for first page##
img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
#Convert to json via CLOVA
Json=Clova(Year,Page)

#Find X coordinate of 'Office'.
XCoord_Unit=Detect_Office(Json,Office)
#Add to motherframe
dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
dta[str(Year)][ExDept][ExOffice]["Ending_Page"]=Page
dta[str(Year)][ExDept][ExOffice]["Office_X2"]=XCoord_Unit+10
dta[str(Year)][ExDept][ExOffice]["Page_Range"]=list(range(ExPage,Page+1))
dta[str(Year)][Dept]

文書課
{'1934': {'Admin': {'文書課': {'Starting_Page': 3}, '職員課': {'Starting_Page': 2}, '庶務課': {'Starting_Page': 5, 'Office_X1': 429}}, '中央卸売市場': {}, '保健局': {}, '土木局': {}, '教育局': {}, '水道局': {}, '港湾部（S8.11.1）': {}, '産業局': {}, '監査局': {}, '社会局': {}, '財務局': {}, '電気局(S8.11.6)': {}, '電気研究所（S8.11.18）': {}}}


{'文書課': {'Starting_Page': 3, 'Office_X1': 292},
 '職員課': {'Starting_Page': 2,
  'Ending_Page': 3,
  'Office_X2': 302,
  'Page_Range': [2, 3]},
 '庶務課': {'Starting_Page': 5, 'Office_X1': 429}}

In [7]:
#Test code| Version 2#
#Show Working office#
n=3

#Extract key info of office
Row  = df.iloc[n]
ExRow= df.iloc[n-1]
Row

Office    庶務課
Dept      監査局
Year        9
Page        5
Name: 3, dtype: object

In [8]:
Page=int(Row["Page"])
Office=Row["Office"]
print(Office)
ExPage=int(ExRow["Page"])
ExOffice=ExRow["Office"]


###Insert Starting page information to motherframe###
dta[Year][Dept][Office]={}
dta[Year][Dept][Office]["Starting_Page"]=Page
print(dta)

###Collect Location information###
##Read image for first page##
img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
#Convert to json via CLOVA
Json=Clova(Year,Page)

#Find X coordinate of 'Office'.
XCoord_Unit=Detect_Office(Json,Office)
#Add to motherframe
dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
dta[str(Year)][ExDept][ExOffice]["Ending_Page"]=Page
dta[str(Year)][ExDept][ExOffice]["Office_X2"]=XCoord_Unit+10
dta[str(Year)][ExDept][ExOffice]["Page_Range"]=list(range(ExPage,Page+1))
dta

庶務課
{'1934': {'Admin': {'文書課': {'Starting_Page': 3, 'Office_X1': 292}, '職員課': {'Starting_Page': 2, 'Ending_Page': 3, 'Office_X2': 302, 'Page_Range': [2, 3]}, '庶務課': {'Starting_Page': 5}}, '中央卸売市場': {}, '保健局': {}, '土木局': {}, '教育局': {}, '水道局': {}, '港湾部（S8.11.1）': {}, '産業局': {}, '監査局': {}, '社会局': {}, '財務局': {}, '電気局(S8.11.6)': {}, '電気研究所（S8.11.18）': {}}}


{'1934': {'Admin': {'文書課': {'Starting_Page': 3,
    'Office_X1': 292,
    'Ending_Page': 5,
    'Office_X2': 439,
    'Page_Range': [3, 4, 5]},
   '職員課': {'Starting_Page': 2,
    'Ending_Page': 3,
    'Office_X2': 302,
    'Page_Range': [2, 3]},
   '庶務課': {'Starting_Page': 5, 'Office_X1': 429}},
  '中央卸売市場': {},
  '保健局': {},
  '土木局': {},
  '教育局': {},
  '水道局': {},
  '港湾部（S8.11.1）': {},
  '産業局': {},
  '監査局': {},
  '社会局': {},
  '財務局': {},
  '電気局(S8.11.6)': {},
  '電気研究所（S8.11.18）': {}}}

In [9]:
dta

{'1934': {'Admin': {'文書課': {'Starting_Page': 3,
    'Office_X1': 292,
    'Ending_Page': 5,
    'Office_X2': 439,
    'Page_Range': [3, 4, 5]},
   '職員課': {'Starting_Page': 2,
    'Ending_Page': 3,
    'Office_X2': 302,
    'Page_Range': [2, 3]},
   '庶務課': {'Starting_Page': 5, 'Office_X1': 429}},
  '中央卸売市場': {},
  '保健局': {},
  '土木局': {},
  '教育局': {},
  '水道局': {},
  '港湾部（S8.11.1）': {},
  '産業局': {},
  '監査局': {},
  '社会局': {},
  '財務局': {},
  '電気局(S8.11.6)': {},
  '電気研究所（S8.11.18）': {}}}

In [184]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)